In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip3 install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 16.3 MB/s 
     |████████████████████████████████| 365 kB 88.4 MB/s 
     |████████████████████████████████| 6.6 MB 58.8 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 596 kB 89.0 MB/s 
     |████████████████████████████████| 212 kB 83.6 MB/s 
     |████████████████████████████████| 141 kB 72.4 MB/s 
     |████████████████████████████████| 127 kB 88.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the 

In [ ]:
!pip3 install tokenizers wandb sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 14.5 MB/s 
     |████████████████████████████████| 1.2 MB 70.3 MB/s 
     |████████████████████████████████| 181 kB 84.4 MB/s 
     |████████████████████████████████| 156 kB 77.1 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=0d76e21425c4ec8d95fdfb8a3fe12a9277f9fad8ef2c9726bc3305de860e373e
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Experiment')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jul 29 19:03:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# ====================================================
# CFG
# ====================================================

class CFG:
    wandb=True
    competition='Upstage'
    _wandb_kernel='bluehills'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    model="klue/roberta-large"
    path="./upstage-roberta-large-lstm-outputs/"
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=50
    batch_size=8
    fc_dropout=0.1
    target_size=1 
    max_len=512
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=False
    use_rnn=False
    bidirectional=False


if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [ ]:
OUTPUT_DIR = './upstage-inference/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    import wandb
    try:
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        # wandb.login(key=secret_value_0)
        print('login to wandb')
        wandb.login()
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='Upstage-Inference', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="inference",
                     anonymous=anony)

login to wandb


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bluehills. Use `wandb login --relogin` to force relogin


# Library

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
from joblib import Parallel, delayed
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm

# Sklearn Imports
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_curve, auc

# PyTorch
import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter, LSTM
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.checkpoint import checkpoint
import torch.cuda.amp as amp


import pickle as pickle


# For Transformer Models
import transformers
from transformers import DataCollatorWithPadding

import tokenizers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, AdamW, AutoModelForSequenceClassification, AutoModelForTokenClassification
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__: 1.12.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.21.0
env: TOKENIZERS_PARALLELISM=true


In [ ]:
from datasets import Dataset, load_metric

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=CFG.seed)

In [ ]:
def klue_re_micro_f1(preds, labels):
    """KLUE-RE micro f1 (except no_relation)"""
    label_list = ['no_relation', 'org:top_members/employees', 'org:members',
       'org:product', 'per:title', 'org:alternate_names',
       'per:employee_of', 'org:place_of_headquarters', 'per:product',
       'org:number_of_employees/members', 'per:children',
       'per:place_of_residence', 'per:alternate_names',
       'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
       'per:spouse', 'org:founded', 'org:political/religious_affiliation',
       'org:member_of', 'per:parents', 'org:dissolved',
       'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
       'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
       'per:religion']
    no_relation_label_idx = label_list.index("no_relation")
    label_indices = list(range(len(label_list)))
    label_indices.remove(no_relation_label_idx)
    return f1_score(labels, preds, average="micro", labels=label_indices) * 100.0

def klue_re_auprc(probs, labels):
    """KLUE-RE AUPRC (with no_relation)"""
    labels = np.eye(30)[labels]

    score = np.zeros((30,))
    for c in range(30):
        targets_c = labels.take([c], axis=1).ravel()
        preds_c = probs.take([c], axis=1).ravel()
        precision, recall, _ = precision_recall_curve(targets_c, preds_c)
        score[c] = auc(recall, precision)
    return np.average(score) * 100.0

def compute_metrics(preds, probs, labels):
    """ validation을 위한 metrics function """
    # labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    # probs = pred.predictions

    # calculate accuracy using sklearn's function
    f1 = klue_re_micro_f1(preds, labels)
    auprc = klue_re_auprc(probs, labels)
    acc = accuracy_score(labels, preds) # 리더보드 평가에는 포함되지 않습니다.

    return {
        'micro f1 score': f1,
        'auprc' : auprc,
        'accuracy': acc,
    }


def label_to_num(label):
    num_label = []
    with open('./code/dict_label_to_num.pkl', 'rb') as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label


def num_to_label(label):
    """
      숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
    """
    origin_label = []
    with open('./code/dict_num_to_label.pkl', 'rb') as f:
      dict_num_to_label = pickle.load(f)
    for v in label:
      origin_label.append(dict_num_to_label[v])
    
    return origin_label

# Tokenizer

In [ ]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.path + 'tokenizer', use_fast=True)
CFG.tokenizer = tokenizer

# Data Loading

In [ ]:
def preprocessing_dataset(dataset):
    """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
    subject_entity = []
    object_entity = []
    for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
        i = i[1:-1].split(',')[0].split(':')[1]
        j = j[1:-1].split(',')[0].split(':')[1]

        subject_entity.append(i)
        object_entity.append(j)
    out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],})
    return out_dataset


def load_data(dataset_dir):
    """ csv 파일을 경로에 맡게 불러 옵니다. """
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_dataset(pd_dataset)
    
    return dataset


def tokenized_dataset(dataset, tokenizer):
    """ tokenizer에 따라 sentence를 tokenizing 합니다."""
    concat_entity = []
    for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
        temp = ''
        temp = e01 + '[SEP]' + e02
        concat_entity.append(temp)
    tokenized_sentences = tokenizer(
        concat_entity,
        list(dataset['sentence']),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
        add_special_tokens=True,
    )
    return tokenized_sentences

In [ ]:
train = load_data("./dataset/train/train.csv")
test = load_data("./dataset/test/test_data.csv")

In [ ]:
display(train.head())
print(train.info())
print(train.shape)
train.describe()

,id,sentence,subject_entity,object_entity,label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,'비틀즈','조지 해리슨',no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,'민주평화당','대안신당',no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'광주FC','한국프로축구연맹',org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,'아성다이소','박정부',org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,'요미우리 자이언츠','1967',no_relation


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32470 entries, 0 to 32469
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              32470 non-null  int64 
 1   sentence        32470 non-null  object
 2   subject_entity  32470 non-null  object
 3   object_entity   32470 non-null  object
 4   label           32470 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB
None
(32470, 5)


,id
count,32470.000000
mean,16234.500000
std,9373.425957
min,0.000000
25%,8117.250000
50%,16234.500000
75%,24351.750000
max,32469.000000


In [ ]:
display(test.head())
print(test.info())
print(test.shape)
test.describe()

,id,sentence,subject_entity,object_entity,label
0,0,지난 15일 MBC '탐사기획 스트레이트'가 이 사실을 보도했다.,'MBC','탐사기획 스트레이트',100
1,1,사랑스러운 ‘프린세스 프링’의 이름은 봄의 공주님: Princess(s)Pring이...,'프린세스 프링','공주',100
2,2,"한편, 본인(이근안)을 모델로 한 MBC 특집드라마 가 1995년 6.25 특집극으...",'경찰','1995년',100
3,3,정창손은 김질과 같이 대궐로 달려가 고변하며 '신은 실로 모르고 김질만 혼자 참여하...,'세조','정창손',100
4,4,"당시 민주당 이진련 시의원은 권영진 시장에게 ""긴급 생계자금을 왜 현금으로 지원하지...",'민주당','권영진',100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7765 entries, 0 to 7764
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              7765 non-null   int64 
 1   sentence        7765 non-null   object
 2   subject_entity  7765 non-null   object
 3   object_entity   7765 non-null   object
 4   label           7765 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 303.4+ KB
None
(7765, 5)


,id,label
count,7765.000000,7765.0
mean,3882.000000,100.0
std,2241.706753,0.0
min,0.000000,100.0
25%,1941.000000,100.0
50%,3882.000000,100.0
75%,5823.000000,100.0
max,7764.000000,100.0


In [ ]:
def load_test_dataset(dataset_dir, tokenizer):
    """
      test dataset을 불러온 후,
      tokenizing 합니다.
    """
    test_dataset = load_data(dataset_dir)
    test_label = list(map(int,test_dataset['label'].values))
    # tokenizing dataset
    tokenized_test = tokenized_dataset(test_dataset, tokenizer)
    return test_dataset['id'], tokenized_test, test_label

# Dataset

In [ ]:
class RE_Dataset(torch.utils.data.Dataset):
    """ Dataset 구성을 위한 class."""
    def __init__(self, cfg, pair_dataset, labels):
      self.cfg = cfg
      self.pair_dataset = pair_dataset
      self.labels = labels

    def __getitem__(self, idx):
      item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
      labels = torch.tensor(self.labels[idx])
      return item, labels

    def __len__(self):
      return len(self.labels)

# Model

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
    # def forward(self, last_hidden_state):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        # sum_embeddings = torch.sum(last_hidden_state, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [ ]:
# class CustomModel(nn.Module):
#     def __init__(self, cfg, config_path=None, pretrained=False):
#         super().__init__()
#         self.cfg = cfg
#         if config_path is None:
#             self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
#         else:
#             self.config = torch.load(config_path)
#         if pretrained:
#             self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
#             # self.model = AutoModelForSequenceClassification.from_pretrained(cfg.model, config=self.config)
#         else:
#             self.model = AutoModel.from_config(self.config)
#             # self.model = AutoModelForSequenceClassification(self.config)
#         self.fc_dropout = nn.Dropout(cfg.fc_dropout)
#         self.fc = nn.Linear(self.config.hidden_size, 30)
#         self._init_weights(self.fc)
        
#     def _init_weights(self, module):
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.Embedding):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.padding_idx is not None:
#                 module.weight.data[module.padding_idx].zero_()
#         elif isinstance(module, nn.LayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)
        
#     def feature(self, inputs):
#         outputs = self.model(**inputs)
#         outputs = outputs.last_hidden_state[:, 0, :]
#         outputs = torch.squeeze(outputs)
#         # last_hidden_states = outputs[0]
#         return outputs

#     def forward(self, inputs):
#         feature = self.feature(inputs)
#         output = self.fc(self.fc_dropout(feature))
#         return output

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False, use_rnn=False, bidirectional=True):
        super().__init__()
        self.cfg = cfg
        self.use_rnn = use_rnn
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
            # self.model = AutoModelForSequenceClassification.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
            # self.model = AutoModelForSequenceClassification(self.config)
        dim = self.config.hidden_size
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(dim, 30)
        self._init_weights(self.fc)

        # check if rnn is used
        if self.use_rnn:
            self.bidirectional = bidirectional
            self.pooling = MeanPooling()
            self.seq_encoder = LSTM(dim, int(dim / 2), 1, dropout=0.1, bidirectional=self.bidirectional, batch_first=True)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[:, 0, :]
        outputs = torch.squeeze(outputs)
        # last_hidden_states = outputs[0]
        return outputs
    
    def encode(self, inputs):
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state
        outputs, _ = self.seq_encoder(outputs)
        return outputs


    def forward(self, inputs):
        if self.use_rnn:
            features = self.encode(inputs)
            features = self.pooling(features, inputs['attention_mask'])
        else:
            features = self.feature(inputs)
        output = self.fc(self.fc_dropout(features))
        return output


# Inference

In [ ]:
# # ====================================================
# # inference
# # ====================================================
# def inference(model, tokenized_sent, device):
#     """
#       test dataset을 DataLoader로 만들어 준 후,
#       batch_size로 나눠 model이 예측 합니다.
#     """
#     dataloader = DataLoader(tokenized_sent, batch_size=16, shuffle=False)
#     model.eval()
#     output_pred = []
#     output_prob = []
#     for i, data in enumerate(tqdm(dataloader)):
#         with torch.no_grad():
#             outputs = model(
#                 input_ids=data['input_ids'].to(device),
#                 attention_mask=data['attention_mask'].to(device),
#                 token_type_ids=data['token_type_ids'].to(device)
#             )
#         logits = outputs[0]
#         prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
#         logits = logits.detach().cpu().numpy()
#         result = np.argmax(logits, axis=-1)

#         output_pred.append(result)
#         output_prob.append(prob)
    
#     return np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

In [ ]:
# ====================================================
# inference
# ====================================================

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    start = end = time.time()
    for step, (inputs, labels) in enumerate(test_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            logits = model(inputs)
        preds.append(logits.to('cpu').numpy())

        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(test_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  .format(step, len(test_loader),
                          remain=timeSince(start, float(step+1)/len(test_loader))))
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
tokenized_test = tokenized_dataset(test, CFG.tokenizer)
labels = test['label'].values
test_dataset = RE_Dataset(CFG, tokenized_test, labels)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.path+"config.pth", pretrained=False, use_rnn=CFG.use_rnn, bidirectional=CFG.bidirectional)
    state = torch.load(CFG.path + f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction
    gc.collect()
    torch.cuda.empty_cache()
predictions = np.mean(predictions, axis=0)

EVAL: [0/971] Elapsed 0m 3s (remain 55m 0s) 
EVAL: [100/971] Elapsed 0m 13s (remain 1m 55s) 
EVAL: [200/971] Elapsed 0m 23s (remain 1m 30s) 
EVAL: [300/971] Elapsed 0m 33s (remain 1m 14s) 
EVAL: [400/971] Elapsed 0m 43s (remain 1m 1s) 
EVAL: [500/971] Elapsed 0m 53s (remain 0m 50s) 
EVAL: [600/971] Elapsed 1m 3s (remain 0m 39s) 
EVAL: [700/971] Elapsed 1m 13s (remain 0m 28s) 
EVAL: [800/971] Elapsed 1m 23s (remain 0m 17s) 
EVAL: [900/971] Elapsed 1m 33s (remain 0m 7s) 
EVAL: [970/971] Elapsed 1m 40s (remain 0m 0s) 
EVAL: [0/971] Elapsed 0m 0s (remain 5m 23s) 
EVAL: [100/971] Elapsed 0m 10s (remain 1m 29s) 
EVAL: [200/971] Elapsed 0m 20s (remain 1m 18s) 
EVAL: [300/971] Elapsed 0m 30s (remain 1m 7s) 
EVAL: [400/971] Elapsed 0m 40s (remain 0m 57s) 
EVAL: [500/971] Elapsed 0m 50s (remain 0m 47s) 
EVAL: [600/971] Elapsed 1m 0s (remain 0m 37s) 
EVAL: [700/971] Elapsed 1m 10s (remain 0m 27s) 
EVAL: [800/971] Elapsed 1m 20s (remain 0m 17s) 
EVAL: [900/971] Elapsed 1m 30s (remain 0m 7s) 
EVAL:

In [ ]:
output_pred = []
output_prob = []

for p in predictions:
    logits = torch.Tensor(p)
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result.tolist())
    output_prob.append(prob.tolist())

In [ ]:
pred_answer = num_to_label(output_pred)
test_id = test['id']

output = pd.DataFrame({'id':test_id,'pred_label':pred_answer,'probs':output_prob,})
display(output.head())
output.to_csv('./submission.csv', index=False) # 최종적으로 완성된 예측한 라벨 csv 파일 형태로 저장.

,id,pred_label,probs
0,0,org:product,"[0.00539395771920681, 0.002790523925796151, 0...."
1,1,per:alternate_names,"[0.009685811586678028, 0.007681087590754032, 0..."
2,2,no_relation,"[0.9149157404899597, 0.002953886752948165, 0.0..."
3,3,no_relation,"[0.9147962927818298, 0.002823289018124342, 0.0..."
4,4,no_relation,"[0.9137780666351318, 0.0037772059440612793, 0...."
